In [1]:
# Importacion de librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import KFold

In [2]:
def readData(rutaArchivo, formato='csv'):
    if formato == 'csv':
        data = pd.read_csv(rutaArchivo)
    elif formato == 'arff':
        data = arff.loadarff(rutaArchivo)
        data = pd.DataFrame(data[0])
    return data

def getNombreClases(data, nombreClase, formato='csv'):
    nombreClases = data[nombreClase].unique()
    if formato == 'arff':
        nombreClases = [clase.decode('utf-8') if isinstance(clase, bytes) else clase for clase in nombreClases]
    return nombreClases

In [3]:
# def getImpurezaGini(data, clases):
#     tipoDato = data.dtypes.name

#     if tipoDato == 'f':
#         sorted_data = data.sort_values()
#         total_count = len(sorted_data)
#         gini_values = []
#         average_values = []

#         for i in range(1, total_count):

    

#     return tipoDato
    

In [94]:
def gini_columna(counts):
        total = counts.sum()
        p = counts / total
        return 1 - np.sum(p ** 2)

def getImpurezaGini(data, clases):
    """
    Calcula la impureza de Gini para un conjunto de datos dado.
    Parámetros:
    @data: DataFrame de pandas que contiene la frecuencia de cada clase.
    @clases: Lista de nombres de las clases.

    Retorna:
    @gini: Valor de la impureza de Gini.
    """

    # def giniSimple(dataClase):
    #     total = 0
    #     for i, xi in enumerate(dataClase[:-1], 1):
    #         total += np.sum(np.abs(xi - dataClase[i:]))
    #     gini = total / (len(dataClase) ** 2 * np.mean(dataClase))
    #     return gini


    # # total = len(data)
    # n, m = data.shape
    # print("Data shape: ", n, m)
    # giniHojas = []


    
    # for clase in range(m):
    #     sum = 0
    #     dataClase = data.iloc[:, clase].values
    #     print("Data clase \n", dataClase)
    #     total = len(dataClase)

    #     giniClase = giniSimple(dataClase)
    #     print("Gini clase ", clases[clase], ": ", giniClase)

    #     giniHojas.append(giniClase)

    # print("Gini hojas: ", giniHojas)
    gini_por_hoja = data.apply(gini_columna, axis=0)

    # --- Pesos (proporción de elementos por hoja) ---
    tamaños = data.sum(axis=0)   # total por hoja
    total = tamaños.sum()

    pesos = tamaños / total

    # --- Gini ponderada ---
    gini_ponderada = (pesos * gini_por_hoja).sum()

    return gini_por_hoja, gini_ponderada

In [ ]:
# Main
data = readData('iris.csv', formato='csv')
c = 1
print(data.head())

print("--------------------")
print(data.columns[c])

print("********************")
print(data.iloc[:, c].dtype)

print("********************")
print(data.iloc[:, c])

print("--------------------")

clasesNombre = getNombreClases(data, "Species", formato='csv')
print("Clases: ", clasesNombre)




# gini = getImpurezaGini(data.iloc[:, c], clasesNombre)
# print(gini)

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa
--------------------
SepalLengthCm
********************
float64
********************
0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: SepalLengthCm, Length: 150, dtype: float64
--------------------
Clases:  ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [36]:
def getFrequencyCounts(data, clases):
    frequency_counts = {}
    values = data.iloc[:, 0].unique()

    # for clase in clases:
    #     count = np.sum(data.iloc[:, -1] == clase)
    #     frequency_counts[clase] = int(count)

    # for x_val in data["X"].unique():
    #     subdf = data[data["X"] == x_val]
    #     frecuencias = subdf["Clase"].value_counts().to_dict()
    #     frequency_counts[x_val] = frecuencias

    # for x_val in clases:
    #     subdf = data[data[data.columns[0]] == x_val]
    #     # print(subdf)
    #     frecuencias = subdf[data.columns[-1]].value_counts().to_dict()
    #     frequency_counts[x_val] = frecuencias

    for x_val, clase_val in zip(data.iloc[:, 0], data.iloc[:, -1]):
        frequency_counts.setdefault(x_val, {})
        frequency_counts[x_val].setdefault(clase_val, 0)
        frequency_counts[x_val][clase_val] += 1

    frequency_counts = {
        x: {c: frequency_counts[x][c] for c in sorted(frequency_counts[x])}
        for x in sorted(frequency_counts)
    }

    return frequency_counts

In [93]:
# Ejemplo


gustoCine = {
    "Id": [1, 2, 3, 4, 5, 6, 7],
    "Palomitas": ["Sí", "Sí", "No", "No", "Sí", "Sí", "No"],
    "Malteada": ["Sí", "No", "Sí", "Sí", "Sí", "No", "No"],
    "Edad": [7, 12, 18, 35, 38, 50, 83],
    "Cine": ["No", "No", "Sí", "Sí", "Sí", "No", "No"]   
}

data = pd.DataFrame(gustoCine)
c = 1
# print(data.head())

# print("--------------------")
# print(data.columns[c])

# print("********************")
# print(data.iloc[:, c].dtype)

# print("********************")
# print(data.iloc[:, c])

# print("--------------------")

clasesNombre = getNombreClases(data, "Cine", formato='csv')
print("Clases: ", clasesNombre)

data.drop(columns=["Id"], inplace=True)

nAtributos = data.columns[:-1]
clase = data.columns[-1]

# Creacion de nuevos dataframes por atributo, con la columna de clase
for atributo in nAtributos:
    # print("Atributo: ", nombre)
    # print("Datos atributo: ", data[nombre])
    frecuencias = {}
    df_fq = pd.DataFrame()

    nuevo_df = pd.DataFrame({
        atributo: data[atributo],
        "Clase": data[clase]
    })

    print("--------------------")
    print("Atributo: ", atributo)
    # print("Nuevo DF:\n", nuevo_df)

    # Detectando si es un dato categórico o numérico
    if pd.api.types.is_numeric_dtype(nuevo_df[atributo]):
        print(f"El atributo '{atributo}' es numérico.")
        
    else:
        print(f"El atributo '{atributo}' es categórico.")
        frecuencias = getFrequencyCounts(nuevo_df, clasesNombre)
        df_fq = pd.DataFrame(frecuencias)
        print("Frecuencias: ", frecuencias)
        print("DF Frecuencias:\n", df_fq)
        gini = getImpurezaGini(df_fq, clasesNombre)
        

    
    
    # print("Impureza Gini: ", gini)

Clases:  ['No' 'Sí']
--------------------
Atributo:  Palomitas
El atributo 'Palomitas' es categórico.
Frecuencias:  {'No': {'No': 1, 'Sí': 2}, 'Sí': {'No': 3, 'Sí': 1}}
DF Frecuencias:
     No  Sí
No   1   3
Sí   2   1
Data shape:  2 2
Data clase 
 [1 2]
Gini clase  No :  0.16666666666666666
Data clase 
 [3 1]
Gini clase  Sí :  0.25
Gini hojas:  [np.float64(0.16666666666666666), np.float64(0.25)]
--------------------
Atributo:  Malteada
El atributo 'Malteada' es categórico.
Frecuencias:  {'No': {'No': 3}, 'Sí': {'No': 1, 'Sí': 3}}
DF Frecuencias:
      No  Sí
No  3.0   1
Sí  NaN   3
Data shape:  2 2
Data clase 
 [ 3. nan]
Gini clase  No :  nan
Data clase 
 [1 3]
Gini clase  Sí :  0.25
Gini hojas:  [np.float64(nan), np.float64(0.25)]
--------------------
Atributo:  Edad
El atributo 'Edad' es numérico.
